In [19]:
%env https_proxy=http://172.16.2.250:3128

env: https_proxy=http://172.16.2.250:3128


In [2]:
from bs4 import BeautifulSoup
import requests
from time import sleep

In [3]:
def scrape_from_page(url, category):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find("header", {"class" : "article-title"}).find("h1").get_text()
    content = "".join([elt.get_text() for elt in soup.find("div", {"id" : "article_content"}).find_all("p")])
    summary = soup.find("h2", {"class" : "subline"}).get_text()
    date = soup.time.attrs['datetime'][:10]

    return [title, date, url, category, summary, content]

In [4]:
def get_topic_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    urls = list(set([elt.find("a")["href"] for elt in soup.find_all("article")]))
    return list(filter(lambda k: 'https' in k, urls))

In [5]:
def scrape(page_no, topic):
    URL = "https://www.ibtimes.co.in/" + topic + "page/" + str(page_no)
    url_list = get_topic_pages(requests.get(URL))
    sleep(1)

    data = []

    for url in url_list:
        data.append(scrape_from_page(url, topic))
        sleep(1)

    return data

In [6]:
def scrape_loop(start_pg, end_pg, topic):
    data = []
    for pg in range(start_pg, end_pg):
        data += scrape(pg, topic)
        
    return data

In [7]:
dt = scrape_loop(1, 10, "science")

SSLError: HTTPSConnectionPool(host='www.ibtimes.co.in', port=443): Max retries exceeded with url: /sciencepage/1 (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))

In [48]:
len(dt)

45

# Adding to database

In [22]:
import sqlite3

In [29]:
database = "news_db.sqlite"
table_name = "articles"

In [30]:
conn = sqlite3.connect(database)

In [40]:
create = f'CREATE TABLE {table_name} (title VARCHAR, date DATE, url VARCHAR, category VARCHAR, summary VARCHAR, content VARCHAR)'
delete = f'DROP TABLE {table_name}'

def run(conn, query):
    conn.cursor().execute(query)
    conn.commit()
    
def table_exists(conn, table_name):
    return conn.cursor().execute(
  f"""SELECT name FROM sqlite_master WHERE type='table'
  AND name='{table_name}'; """).fetchall() != []

def show_table(conn, table_name):
    return conn.execute(f"SELECT * FROM {table_name}").fetchall()
    
def insert_data(conn, table_name, data):
    insert = f"""
    INSERT INTO {table_name}
    (title, date, url, category, summary, content)
    VALUES (?, ?, ?, ?, ?, ?)
    """
    conn.cursor().executemany(insert, data)
    conn.commit()

In [43]:
# run(conn, delete)

In [44]:
if not (table_exists(conn, "articles")): run(conn, create)

In [45]:
insert_data(conn, "articles", dt)

In [46]:
show_table(conn, "articles")

[('Why HDFC Limited will be merged with HDFC Bank: Explained',
  '2022-04-04',
  'https://www.ibtimes.co.in/why-hdfc-limited-will-be-merged-hdfc-bank-explained-847163',
  'science',
  "HDFC Limited, India's largest housing finance company with Rs 5.26 trillion in assets under management (AUM) and a market capitalization of Rs 4.44 trillion, will merge with HDFC Bank, India's largest private sector bank by assets with a market capitalization of Rs 8.35 trillion.",
  'HDFC Ltd, India\'s largest housing financing firm, will merge with HDFC Bank, the country\'s largest private lender, to form a banking behemoth in one of the largest mergers in corporate history. According to stock exchange filings by the companies, HDFC Bank would be 100% held by public shareholders once the merger is completed, and existing HDFC shareholders will own 41% of the bank. For every 25 HDFC Bank shares owned, each HDFC shareholder will receive 42 HDFC Bank shares."This is a merger of equals. We believe that the

In [47]:
conn.close()

In [51]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk import ngrams

import textcleaner as tc

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saksh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saksh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saksh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [81]:
cleaned_docs = [(str(tc.document(elt[5]).lower_all().remove_symbols().remove_stpwrds().lemming().stemming()),) for elt in dt]

In [77]:
cleaned_docs[1]

(lend major axi bank acquir citibank consum busi india subject regulatori approvalsth board axi bank citibank approv deal compris sale consum busi citibank india includ credit card retail bank wealth manag consum loan well sale consum busi citi nonbank financi compani citicorp financ india ltd compris assetback financ busi includ commerci vehicl construct equip loan well person loan portfolioaxi bank look acquisit healthi strateg fit itt said statementit gain access larg affluent custom franchis citibank bouquet feeorient profit segment includ qualiti credit card portfolio affluent wealth manag clientel meaning deposit 81 per cent casa along strong consum lend portfoliopost acquisit axi bank 285 million save account 23 lakh burgundi custom 106 million cardswith requir capabl place expect success integr across key paramet includ employe custom product offer technolog valu accret manneraddit deal offer strateg advantag bank premiumis overal custom portfolio increas opportun crosssel prod